In [2]:
import sys
sys.path.insert(0, '..')
from net_framework import *
import pandas as pd

In [3]:
term_data = pd.read_csv('term.txt', sep="\t", header=None)
term_data.columns = ["#Lnum", "#snum", "#cnum", "Term Abbrev"]
term_data.head()

,#Lnum,#snum,#cnum,Term Abbrev
0,1,1,1,LB
1,1,1,2,LB
2,1,1,3,LE
3,1,1,4,WK
4,1,1,5,LF


In [10]:
cnum_data = pd.read_csv('cnum-vhcm-lab-new.txt', sep="\t")
locations = cnum_data[['#cnum']]
locations['Normalized-L'] = (cnum_data['L*'] - cnum_data['L*'].mean())/(cnum_data['L*'] - cnum_data['L*'].mean()).max()
locations['Normalized-a'] = (cnum_data['a*'] - cnum_data['a*'].mean())/(cnum_data['a*'] - cnum_data['a*'].mean()).std() * 1/2
locations['Normalized-b'] = (cnum_data['b*'] - cnum_data['b*'].mean())/(cnum_data['b*'] - cnum_data['b*'].mean()).std() * 1/2
display(locations.head(5))

,#cnum,Normalized-L,Normalized-a,Normalized-b
0,141,1.000000,-0.025675,-0.138822
1,274,0.876301,-0.025504,-0.138822
2,129,0.876301,0.070445,-0.106039
3,230,0.876301,0.070101,-0.089951
4,302,0.876301,0.070617,-0.072041


In [30]:
chip_num = list(locations['#cnum'])
lab_norm = [[row[2], row[3], row[4]] for row in locations.itertuples()]

In [11]:
unique_symbols = list(term_data['Term Abbrev'].unique())
l1 = term_data[term_data.get('#Lnum').eq(1)]
l1_grouped = l1.groupby('#cnum')['Term Abbrev'].apply(list)
l1_chip_abbrev_percentage = [[(l1_grouped[i + 1].count(abbrev) / len(l1_grouped[i + 1])) for abbrev in unique_symbols] for i in range(len(l1_grouped))]

In [12]:
l1_result = pd.DataFrame(l1_chip_abbrev_percentage)
l1_result.index += 1
l1_result.index.name = '#cnum'
l1_result.columns = unique_symbols
print(l1_result)

         LB    LE    WK    LF     F     G     S   GB    FU   YN  ...   YK  \
#cnum                                                            ...        
1      0.36  0.00  0.00  0.04  0.08  0.52  0.00  0.0  0.00  0.0  ...  0.0   
2      0.12  0.00  0.04  0.12  0.60  0.04  0.00  0.0  0.08  0.0  ...  0.0   
3      0.00  0.96  0.04  0.00  0.00  0.00  0.00  0.0  0.00  0.0  ...  0.0   
4      0.28  0.00  0.40  0.00  0.08  0.04  0.20  0.0  0.00  0.0  ...  0.0   
5      0.04  0.00  0.00  0.20  0.68  0.08  0.00  0.0  0.00  0.0  ...  0.0   
...     ...   ...   ...   ...   ...   ...   ...  ...   ...  ...  ...  ...   
326    0.08  0.20  0.36  0.00  0.20  0.00  0.12  0.0  0.04  0.0  ...  0.0   
327    0.04  0.00  0.00  0.68  0.20  0.04  0.04  0.0  0.00  0.0  ...  0.0   
328    0.64  0.00  0.08  0.00  0.00  0.20  0.08  0.0  0.00  0.0  ...  0.0   
329    0.08  0.08  0.40  0.00  0.12  0.00  0.28  0.0  0.04  0.0  ...  0.0   
330    0.04  0.00  0.00  0.72  0.24  0.00  0.00  0.0  0.00  0.0  ...  0.0   

In [36]:
#Number of training iterations
num_iters = 200
#Size of training dataset
num_examples = 1331
#Listing out the shapes of each model
input_len = len(chip_num)
network_shapes = [(input_len, [200], 1), (input_len, [100], 1), (input_len, [50], 1)]
#Learning rate of the network
rate = 7

#Generating Training Data
input_num = torch.FloatTensor(lab_norm)
output_num = torch.FloatTensor(chip_num)

#Array of losses over training period for each network
for net_num, shape in enumerate(network_shapes):
    print('Network Shape:',flush = True)
    print('Input Size = ' + str(shape[0]), flush = True)
    print('Hidden Size = ' + str(shape[1]), flush = True)
    print('Output Size = ' + str(shape[2]), flush = True)
    NN = Neural_Network(inputSize = shape[0], outputSize = shape[2],
                        hiddenSize = shape[1] , learning_rate = rate)
    

    for i in range(num_iters):
        #Calculating l1 error
        error = NN.l1error(output_num, NN(input_num))
        if i == 0: 
            dh = display("#" + str(i) + " Error: " + str(error), display_id=True)
        else:
            dh.update("#" + str(i) + " Error: " + str(error))
            
        NN.train(input_num, output_num)
    #Saves the training results in a filed named "Net 0", "Net 1", etc. 
    NN.saveWeights(model = NN, path = "saved_networks/Net " + str(net_num));

Network Shape:
Input Size = 330
Hidden Size = [3]
Output Size = 1


RuntimeError: size mismatch, m1: [330 x 3], m2: [330 x 3] at ../aten/src/TH/generic/THTensorMath.cpp:41